In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os
from natsort import natsorted



### def read_image_directory()
directory = 'PIVimages/crosscorrelation'
prefix = None
type = 'png'

# Get a list of files in the directory
files = os.listdir(directory)

# If a prefix is specified, filter the list of files
if prefix is not None:
    files = [f for f in files if f.startswith(prefix)]
    
# If a type is specified, filter the list of files
if type is not None:
    files = [f for f in files if f.endswith(type)]
    
# Sort the files
files = natsorted(files)

# Read the images and store them in a list
images = [cv.imread(os.path.join(directory, f), 
                    cv.IMREAD_GRAYSCALE) for f in files]

In [ ]:
# Count the number of images, make sure only two go to the next function